Import related libraries

In [1]:
import os.path

import openai
# sokoban_solver is a custom package that contains the A* agent that is able to solve Sokoban environment levels
from src.utilities import *
import random

You must add your own OpenAI API key to the `openai-api-key.txt` file.

In [2]:
api_key = get_file_contents("openai-api-key.txt")

Initialize the OpenAI API Client with the API key

In [3]:
openai.api_key = api_key

Set your training mode for the fine-tuning process, all files related to this mode will have the name of the mode in their names.

In [20]:
train_mode = "10-input-3-epoch-0.5-temp-50-output"

Get hyperparameters for the fine-tuning process

In [21]:
hypers = read_hypers()

In [22]:
print(hypers)

{'10-input-3-epoch-0.5-temp-50-output': {'training_set': 'training-set-10-examples', 'n_epochs': 3, 'temperature': 0.5, 'n_generation': 50}}


Upload the training file to the API for fine-tuning purposes

In [23]:
training_data_path = f"data/training_data/{hypers[train_mode]["training_set"]}.jsonl"

with open(training_data_path, "r") as file:
    response = openai.File.create(file=file, purpose="fine-tune")

file_id = response["id"]
write_file_contents(f"data/client_file_ids/{train_mode}.txt", file_id)

FileNotFoundError: [Errno 2] No such file or directory: 'data/training_data/training-set-10-examples.jsonl'

Set the model to fine-tune. gpt-3.5-turbo-0125 is the currently recommended model.

In [24]:
model = "gpt-3.5-turbo-0125"

Create a fine-tuning job for the model with the training file

In [25]:
training_file_id = get_file_contents(f"data/client_file_ids/{train_mode}.txt")

response = openai.FineTune.create_job(
    model=model,
    training_file_id=training_file_id,
    suffix=train_mode,
    n_epochs=hypers[train_mode]["n_epochs"]
)

fine_tuning_job_id = response["id"]
write_file_contents(f"data/fine_tuning_job_ids/{train_mode}.txt", fine_tuning_job_id)

FileNotFoundError: [Errno 2] No such file or directory: 'data/client_file_ids/10-input-3-epoch-0.5-temp-50-output.txt'

Generate levels using the fine-tuned model

In [ ]:
fine_tuned_model = get_file_contents(f"data/fine_tuning_job_ids/{train_mode}.txt")

for i in range(hypers[train_mode]["n_generations"]):
    m = random.randint(7, 35)
    n = random.randint(8, 35)
    response = openai.Completion.create(
        model=fine_tuned_model,
        prompt=f"Create a Sokoban level with a size of {m}x{n}.",
        n=1
    )
    generated_level = response.choices[0].text
    if not os.path.exists(f"data/generated_levels/{train_mode}"):
        os.makedirs(f"data/generated_levels/{train_mode}")
    write_file_contents(f"data/generated_levels/{train_mode}/level_{i}_{m}x{n}.txt", generated_level)